# **ธนภัทร พูลโภคา 6710450911 หมู่ 200**

In [379]:
import pandas as pd
import numpy as np
import datetime

# 1. เปิดไฟล์ italy_earthquakes_from_2016-08-24_to_2016-11-30.csv

In [380]:
eq = pd.read_csv("italy_earthquakes_from_2016-08-24_to_2016-11-30.csv")
eq

,Time,Latitude,Longitude,Depth/Km,Magnitude
0,2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
1,2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2,2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
3,2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
4,2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...,...
8081,2016-11-30 18:39:27.600,42.8903,13.0197,10.7,2.0
8082,2016-11-30 18:43:14.850,42.9443,13.2003,8.6,2.6
8083,2016-11-30 20:18:27.550,43.0092,13.1288,8.0,2.2
8084,2016-11-30 20:45:11.780,43.0197,13.1017,9.3,2.7


# 2. ปรับคอลัมน์ Time ให้เป็น Timestamp และตั้งให้เป็น index

In [381]:
eq["Time"] = pd.to_datetime(eq["Time"])
eq.set_index("Time", inplace=True)
eq

,Latitude,Longitude,Depth/Km,Magnitude
Time,,,,
2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...
2016-11-30 18:39:27.600,42.8903,13.0197,10.7,2.0
2016-11-30 18:43:14.850,42.9443,13.2003,8.6,2.6
2016-11-30 20:18:27.550,43.0092,13.1288,8.0,2.2


# 3. เวลาที่เกิดแผ่นดินไหวแรงที่สุดคือวันเวลาไหน

In [382]:
mx_datetime = eq.sort_values("Time", ascending=True).iloc[0, :].name
mx_datetime.strftime("%A %d %B %Y %H:%M:%S") + " - Magnitude " + str(eq.loc[mx_datetime, "Magnitude"])

'Wednesday 24 August 2016 03:36:32 - Magnitude 6.0'

# 4. ในวันที่เกิดแผ่นดินไหวแรงที่สุด มีแผ่นดินไหวกี่ครั้ง

In [383]:
eq[eq.index.date == mx_datetime.date()].sort_index()

,Latitude,Longitude,Depth/Km,Magnitude
Time,,,,
2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...
2016-08-24 23:35:25.340,42.7577,13.0885,10.7,2.0
2016-08-24 23:39:06.170,42.6725,13.1917,9.6,2.0
2016-08-24 23:39:30.350,42.6053,13.2318,18.0,2.2


# 5. แสดงข้อมูลตั้งแต่ 7 ถึง 9 โมงเช้าในวันที่เกิดแผ่นดินไหวแรงที่สุด 

In [384]:
lower_mx_datetime = mx_datetime
upper_mx_datetime = mx_datetime
lower_mx_datetime = lower_mx_datetime.replace(hour=7).floor(freq="h")
upper_mx_datetime = upper_mx_datetime.replace(hour=9).floor(freq="h")
ranged_mx_date_eq = eq[(eq.index.date == mx_datetime.date()) & (eq.index.time >= lower_mx_datetime.time()) & (eq.index.time <= upper_mx_datetime.time())]

# 6. แผ่นดินไหวที่แรงที่สุดกับแรงเป็นที่ 2 ห่างกันเท่าไหร่ ตอบเป็นวัน-ชั่วโมง-นาที

In [385]:
diff = eq.sort_values(by="Magnitude").iloc[0].name - eq.sort_values(by="Magnitude").iloc[1].name
print("{:d}-{:02d}-{:02d}".format(diff.components.days, diff.components.hours, diff.components.minutes))

63-09-08


# 7. คำนวณความแรงของแผ่นดินไหวที่เปลี่ยนไปในแต่ละครั้ง ของข้อมูลในข้อ 5.
*หมายเหตุ: ใช้คำสั่ง shift() ช่วยเลื่อนแถวของคอลัมน์ Magnitude เพื่อคำนวณความแตกต่าง*